# Working with Keras a Deep Dive 
- Creating Keras models with the `Sequential` class, the `Function API`, and `Model Subclassing`
- Using built-in **Keras** `Training` and `Evaluation Loops`
- Using **Keras** `callbacks` to customize training
- Using **TensorBoard** to `Monitor training` and `Evaluation Metrics`
- Writing `Training` and `Evaluation Loops` from sratch

In [26]:
import numpy as np

from tensorflow import keras

## 3 different ways to build Keras model

#### The Sequential Model
It's the most user-friendly API; it's just a Python list. Pros: Ideal for inexperienced users. Cons: constrained to simple layer stacks

The Sequential class

In [2]:
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
])

Metal device set to: Apple M1 Pro


2022-03-21 22:56:07.613518: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-21 22:56:07.614014: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Incrementally building a Sequential model

In [3]:
model.add(keras.layers.Dense(10, activation='softmax'))

Build a model

In [4]:
model.build(input_shape=(None, 3))

Checking a model weights

In [5]:
model.weights

[<tf.Variable 'dense/kernel:0' shape=(3, 64) dtype=float32, numpy=
 array([[ 0.27183855,  0.17330763,  0.01112556,  0.2683748 ,  0.22721225,
          0.03010982, -0.03242248, -0.06754811, -0.07674886,  0.2358855 ,
          0.19749796, -0.16703354,  0.27355605, -0.07441002,  0.05687818,
         -0.08073817,  0.00412124, -0.24933016, -0.03261206,  0.28775   ,
         -0.17331432,  0.09671351, -0.19806853, -0.02006662,  0.12556976,
          0.2764969 , -0.21953611,  0.2516665 ,  0.01063269, -0.1568693 ,
          0.03032714, -0.01586881,  0.03502452,  0.11786652, -0.26522866,
          0.19840914, -0.18462738, -0.03719249, -0.03143367,  0.21938276,
          0.13540858, -0.12857784,  0.00113928,  0.25125682,  0.18222633,
         -0.07037717, -0.14721522, -0.01732507, -0.11818559,  0.01164085,
          0.24812746,  0.02095795,  0.0963906 , -0.23441562, -0.13455468,
         -0.27810273, -0.29565233, -0.02478674, -0.26881185,  0.1465235 ,
          0.10194826,  0.13959166,  0.0809969

Summary a model

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                256       
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 10)                330       
                                                                 
Total params: 6,826
Trainable params: 6,826
Non-trainable params: 0
_________________________________________________________________


Naming Each layers

In [7]:
model = keras.Sequential(name='test_model')
model.add(keras.layers.Dense(64, activation='relu', input_shape=(None, 3), name='first_layer'))
model.add(keras.layers.Dense(64, activation='softmax', name='last_layer'))

# summary
model.summary()

Model: "test_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 first_layer (Dense)         (None, None, 64)          256       
                                                                 
 last_layer (Dense)          (None, None, 64)          4160      
                                                                 
Total params: 4,416
Trainable params: 4,416
Non-trainable params: 0
_________________________________________________________________


#### The Functional API

Common ways to build a Model.
\
Simple Functional model with two Dense layers



In [12]:
inputs = keras.Input(shape=(3,), name='inputs')
features = keras.layers.Dense(units=64, activation='relu')(inputs)
outputs = keras.layers.Dense(units=10, activation='sigmoid')(features)

model = keras.Model(inputs, outputs)

In [13]:
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 3)]               0         
                                                                 
 dense_10 (Dense)            (None, 64)                256       
                                                                 
 dense_11 (Dense)            (None, 10)                650       
                                                                 
Total params: 906
Trainable params: 906
Non-trainable params: 0
_________________________________________________________________


Multi-inputs, Multi-outputs Model
\
Building a system to rank customer support ticket by priority and route them to the appropriate department

In [29]:
vocab_size = 10000
num_tags = 100
num_departments = 4

In [23]:
# inputs 

# title of the ticket
title = keras.Input(shape=(vocab_size,), name='title')

# text body of the ticket
text_body = keras.Input(shape=(vocab_size,), name='text_body')

# tags added by the user
tags = keras.Input(shape=(num_tags, ), name='tags')

# features
features = keras.layers.Concatenate()([title, text_body, tags])
features = keras.layers.Dense(units=64, activation='relu')(features)

# outputs
priority = keras.layers.Dense(units=1, activation='sigmoid', name='priority')(features)
department = keras.layers.Dense(units=num_departement, activation='softmax', name='department')(features)

# create the model by specifying its inputs and outputs
model = keras.Model(inputs= [title, text_body, tags], outputs= [priority, department])

In [25]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 title (InputLayer)             [(None, 10000)]      0           []                               
                                                                                                  
 text_body (InputLayer)         [(None, 10000)]      0           []                               
                                                                                                  
 tags (InputLayer)              [(None, 100)]        0           []                               
                                                                                                  
 concatenate_1 (Concatenate)    (None, 20100)        0           ['title[0][0]',                  
                                                                  'text_body[0][0]',        

Training a Multi-input, Multi-output Model

In [30]:
num_samples = 1280

# dummies dataset
title_data = np.random.randint(0, 2, size=(num_samples, vocab_size))
text_body_data = np.random.randint(0, 2, size=(num_samples, vocab_size))
tags_data = np.random.randint(0, 2, size=(num_samples, num_tags))

priority_data = np.random.random(size=(num_samples, 1))
department_data = np.random.randint(0, 2, size=(num_samples, num_departments))

In [31]:
# train model
model.compile(
    optimizer = keras.optimizers.Adam(learning_rate=1e-2),
    loss = ['mean_squared_error', 'categorical_crossentropy'],
    metrics = [['mean_absolute_error'], ['accuracy']]
)

model.fit(
    [title_data, text_body_data, tags_data],
    [priority_data, department_data],
    epochs = 1)

2022-03-21 23:20:34.587438: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-03-21 23:20:34.901106: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


40/40 [==============================] - 3s 12ms/step - loss: 362.8061 - priority_loss: 0.3235 - department_loss: 362.4826 - priority_mean_absolute_error: 0.4873 - department_accuracy: 0.2320


In [32]:
# predict

model.evaluate([title_data, text_body_data, tags_data], [priority_data, department_data])

11/40 [=======>......................] - ETA: 0s - loss: 844.9823 - priority_loss: 0.3444 - department_loss: 844.6378 - priority_mean_absolute_error: 0.5117 - department_accuracy: 0.5398

2022-03-21 23:21:49.318105: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


40/40 [==============================] - 1s 9ms/step - loss: 844.0298 - priority_loss: 0.3277 - department_loss: 843.7021 - priority_mean_absolute_error: 0.4923 - department_accuracy: 0.5641


[844.02978515625,
 0.32774293422698975,
 843.7020874023438,
 0.4923197329044342,
 0.5640625357627869]

Retrieving the inputs or outputs of a layers in a Functional model

In [37]:
model.layers

In [42]:
# retrive input 
model.layers[3].input

[<KerasTensor: shape=(None, 10000) dtype=float32 (created by layer 'title')>,
 <KerasTensor: shape=(None, 10000) dtype=float32 (created by layer 'text_body')>,
 <KerasTensor: shape=(None, 100) dtype=float32 (created by layer 'tags')>]

In [48]:
# build a new model using exist inputs
inputs = model.layers[3].input
features = model.layers[4].output

difficulty = keras.layers.Dense(units=3, activation='softmax', name='difficulty')(features)

new_model = keras.Model(inputs = inputs, outputs = [priority, department, difficulty])

In [50]:
new_model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 title (InputLayer)             [(None, 10000)]      0           []                               
                                                                                                  
 text_body (InputLayer)         [(None, 10000)]      0           []                               
                                                                                                  
 tags (InputLayer)              [(None, 100)]        0           []                               
                                                                                                  
 concatenate_1 (Concatenate)    (None, 20100)        0           ['title[0][0]',                  
                                                                  'text_body[0][0]',        